In [ ]:
!pip install haversine

In [ ]:
from tqdm import tqdm
import pandas as pd
import plotly.express as px
import csv
import haversine

tqdm.pandas()

# I. Get files from simplimaps and NOAA

## 1. List of all NOAA GHCN stations
This file is in ASCII format so we will convert it into a DataFrame and also save it for future use.

In [ ]:
!wget ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt

In [ ]:
colspecs = [(0,11), (12,20), (21,30), (31,37), (38,40), (41,71), (72,75), (76,79), (80,85)]
stations = pd.read_fwf('ghcnd-stations.txt', colspecs=colspecs, header=None, index_col=None)
stations.columns = ["ID", "LATITUDE", "LONGITUDE", "ELEVATION", "STATE", "NAME", "GSNFLAG", "HCNFLAG", "WMOID"]
stations.reset_index(inplace=True)
stations["COORDS"] = list(zip(stations.LATITUDE, stations.LONGITUDE)) #Savinf coordinates as a tuple will be useful later
stations.to_csv('ghcnd-stations.csv', quoting=csv.QUOTE_NONNUMERIC)

## 2. GHCN Daily Weather data files
These files are available as zipped archives for each year. The zipped archives contain a CSV file without any header. 

In [ ]:
!wget ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2020.csv.gz
!wget ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2019.csv.gz
!wget ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2018.csv.gz

In [ ]:
!wget ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2017.csv.gz
!wget ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2016.csv.gz
!wget ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2015.csv.gz

In [ ]:
!gunzip 2020.csv.gz
!gunzip 2019.csv.gz
!gunzip 2018.csv.gz

In [ ]:
!gunzip 2017.csv.gz
!gunzip 2016.csv.gz
!gunzip 2015.csv.gz

## 3. World Cities Data
Data on 26000 cities inculding geo co-ordinates, country, and population. This data is available from https://simplemaps.com/data/world-cities

In [ ]:
!wget https://simplemaps.com/static/data/world-cities/basic/simplemaps_worldcities_basicv1.71.zip

In [ ]:
!unzip ./simplemaps_worldcities_basicv1.71.zip

# II. Finding the closest weather station for each city

1. We will use one helper function to calculate the Haversine distance between two points. We will use this function to calculate distances between each city and all other stations in the NOAA GHCN Station list. The function will return the 'ID' of the closest station and the distance between the city and station.

2. We will limit our analysis to the top 2500 cities by population because of resource constraints of Kaggle Notebooks. For sure, the code below can be used for analysis of a larger number of cities as also custom lists of cities.

In [ ]:
from haversine import haversine

def NearestStations(lat,long):
    yo = stations
    coords = (lat,long)
    yo["Dist"]= yo.COORDS.apply(lambda x: haversine(coords,x))
    #yo.reset_index(inplace=True)
    closest = yo.loc[yo["Dist"].idxmin()]
    return(str(closest.ID),float(closest.Dist))

In [ ]:
world_cities = pd.read_csv("worldcities.csv")

In [ ]:
world_cities['Closest Station ID'], world_cities['Closest Station Distance'] = \
                            zip(*world_cities.progress_apply(lambda x: NearestStations(float(x['lat']),float(x['lng'])), axis=1))

In [ ]:
world_cities.to_csv("World Cities Nearest Stations.csv")

# III. Extracting weather data for stations

1. We will first extract the list of unique weather station from our list of top 2500 cities and their associated weather stations.

2. Next we will iterate over each years weather data. 

3. In each iteration we will extract the data of only those stations appearing in our list. 

4. We will also transform the data to a more readable format using the pivot() function of pandas dataframe

5. The final transformed data will be saved on file and yields our dataset.

In [ ]:
stations_list = list(world_cities["Closest Station ID"].unique())

In [ ]:
years = ['2015','2016','2017','2018','2019','2020']
for y in years:
    print(y)
    weather_temp = pd.read_csv(y+".csv")
    weather_temp.columns = ['ID','Date','Element','Value','A','B','C','D']
    weather_temp2 = weather_temp[weather_temp.ID.isin(stations_list)]
    wp = weather_temp2.pivot(index = ["Date","ID"], columns = "Element", values="Value")
    wp.reset_index(inplace=True)
    wp.to_csv("Weather all cities "+y+".csv")

In [ ]:
test = pd.read_csv("Weather all cities 2018.csv")
test["TAVG"].tail()

# IV. Analysis

## 1. Analysis of distance between cities and closest weather station

i) About 50% of the cities have a weather station within a 7 kilometer range
ii) About 60% of the cities have a station within a 15 kilometer range
iii) Some cities don't seem to have any nearby stations on the NOAA list

In [ ]:
world_cities["Closest Station Distance"].describe()

In [ ]:
100*world_cities[world_cities["Closest Station Distance"]<15].shape[0]/world_cities.shape[0]

In [ ]:
hist = px.histogram(world_cities, "Closest Station Distance")
hist.show()

## 2. How many weather stations from our list were polled by NOAA for weather data?

1. In each year between 2015-2020, NOAA GHCN collected data from about 53-57% of the stations in our list

In [ ]:
unique_stations_lists = []
unique_stations_nos = []

for y in years:
    w = pd.read_csv("Weather all cities "+y+".csv")
    yo = list(w['ID'].unique())
    yo_len = len(yo)
    unique_stations_lists.append(yo)
    unique_stations_nos.append(yo_len)

unique_stations_all = sorted(set(sum(unique_stations_lists, []))) #ID of every station that was polled at least once 

total_unique_stations_nos = len(list(world_cities["Closest Station ID"].unique()))
unique_stations_percentages = [yo * 100/total_unique_stations_nos for yo in unique_stations_nos]

Percentage_Stations_Polled = pd.DataFrame(list(zip(years, unique_stations_percentages)), columns=["Year", "Percentage of Stations Polled"])

In [ ]:
Percentage_Stations_Polled.head(6)

In [ ]:
Cities_Not_Polled = world_cities[-(world_cities["Closest Station ID"].isin(unique_stations_all))]
Cities_Not_Polled.shape

In [ ]:
Cities_Not_Polled["country"].value_counts()

In [ ]:
Cities_Polled = world_cities[(world_cities["Closest Station ID"].isin(unique_stations_all))]
Cities_Polled.shape

In [ ]:
Cities_Not_Polled["country"].value_counts()